# Offside Detection Video Output

In [26]:
# Check if GPU is being used
!nvidia-smi

Mon Nov 25 12:02:49 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   47C    P3             33W /  170W |    3207MiB /  12288MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [27]:
import os
import sys

# Run on GPU
os.environ["ONNXRUNTIME_EXECUTION_PROVIDER"] = "CUDAExecutionProvider"
project_root = os.path.abspath('..')
sys.path.append(project_root)

In [28]:
import supervision as sv
from common.team import TeamClassifier
from tqdm import tqdm
import numpy as np
from inference import get_model
from dotenv import load_dotenv
import torch
import cv2
from sports.annotators.soccer import draw_pitch, draw_points_on_pitch, draw_pitch_voronoi_diagram
from sports.configs.soccer import SoccerPitchConfiguration

In [ ]:
# Setup paths for source & target videos
# Change to dynamic inputs for cloud deployment
# SOURCE_VIDEO_PATH = "../videos/08fd33_4.mp4"
SOURCE_VIDEO_PATH = "../videos/soccer_video_offside_3.mp4"
TARGET_VIDEO_PATH = "08fd33_4_result_1.mp4"
DEVICE = 'cuda'
BALL_ID = 0
GOALKEEPER_ID = 1
PLAYER_ID = 2
REFEREE_ID = 3
OFFSIDE_ID = 4
STRIDE = 30

CONFIG = SoccerPitchConfiguration()

In [30]:
# Load env variables
load_dotenv()
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

# ID for roboflow detection model created
PLAYER_DETECTION_MODEL_ID = "football-players-detection-3zvbc/12"

# Load Player/Ball model
PLAYER_DETECTION_MODEL = get_model(
    model_id=PLAYER_DETECTION_MODEL_ID,
    api_key=ROBOFLOW_API_KEY
)

# Not enough epochs on own model -- grab model better trained model from Roboflow
# https://universe.roboflow.com/roboflow-jvuqo/football-field-detection-f07vi/model/15

# Load Pitch Detection Model
PITCH_DETECTION_MODEL_ID ="football-field-detection-f07vi/15"
PITCH_DETECTION_MODEL = get_model(
    model_id=PITCH_DETECTION_MODEL_ID, 
    api_key=ROBOFLOW_API_KEY
)

In [31]:
# Setup class for displaying Ellipse around players
ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)

# Setup class for displaying triangle over ball
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FF1493'),
    base=20,
    height=17,
)

# Setup class for labeling box annotations
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER,
)

# Create Vertext Annotator for pitch detection
vertex_annotator = sv.VertexAnnotator(
    color=sv.Color.from_hex('#FF1493'),
    radius=8
)

# Create edge annotations using SoccerPitchConfiguration
edge_annotator = sv.EdgeAnnotator(
    color=sv.Color.from_hex('00BFFF'),
    thickness=2,
    edges=CONFIG.edges
)

In [32]:
# Function to extract cropped images from video to identify player teams
# @Params: video_path<str>
# @Return: crops<arr>
def extract_crops(source_video_path: str):

    # Generate frames from source video
    # Stride: Only use every nth frame
    frame_generator = sv.get_video_frames_generator(source_path=SOURCE_VIDEO_PATH, stride=STRIDE)

    crops = []

    # Loop through each frame generated
    for  frame in tqdm(frame_generator, desc="collection crops"):

        # Infer object type (ie. person) with minimum confidence
        # Covert to standard format
        # Remove overlapping detections
        # Filter detections by class --> only keep players
        result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
        detections = sv.Detections.from_inference(result)
        detections = detections.with_nmm(threshold=0.5, class_agnostic=True)
        detections = detections[detections.class_id == PLAYER_ID]
        crops += [
            # Out of the box feature that crops out based on the box coordinates (xyxy)
            sv.crop_image(frame, xyxy)
            for xyxy
            in detections.xyxy
        ]
    
    return crops

In [33]:
def resolve_goalkeepers_team_id(players_detections: sv.Detections, goalkeepers_detections: sv.Detections) -> np.ndarray:
    
    # Players will have 0 or 1 depending on the team they belong to --> from TeamClassifier
    # Get the coordinates of goalkeeper and player detections from the lower center of bounding box
    goalkeepers_xy = goalkeepers_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    players_xy = players_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)

    # Filter out players based on class id (which team they are on)
    # Find the average position of players in each team
    # This can also be used to determine the direction of attack for each team
    team_0_centroid = players_xy[players_detections.class_id == 0].mean(axis=0)
    team_1_centroid = players_xy[players_detections.class_id == 1].mean(axis=0)

    # Loop over goalkeepers
    # Find distance of each goalkeeper from each team centroid
    # Goal keeper will belong to team they are closest to on average
    goalkeepers_team_ids = []
    for goalkeeper_xy in goalkeepers_xy:
        dist_0 = np.linalg.norm(goalkeeper_xy - team_0_centroid)
        dist_1 = np.linalg.norm(goalkeeper_xy - team_1_centroid)
        goalkeepers_team_ids.append(0 if dist_0 < dist_1 else 1)

    return np.array(goalkeepers_team_ids)

In [ ]:
def resolve_offside_player(players_detections: sv.Detections, pitch_players_xy: np.ndarray) -> np.ndarray:
    
    # Get average center positions on the x-axis for each team
    # Lower number defends the left side of the field while attacking the right
    # Higher number defends the right side of the field while attacking the left
    team_0_center = pitch_players_xy[players_detections.class_id == 0][0].mean(axis=0) # Defending
    team_1_center = pitch_players_xy[players_detections.class_id == 1][0].mean(axis=0) # Attacking

    # Ensure team 0 is defending left side --> swap class ids & centers if not
    if team_0_center > team_1_center:
        temp = team_1_center
        team_0_center = team_1_center
        team_1_center = temp

        for i in players_detections.class_id:

            j = 0

            if i == 0:
                j = 1
            if i == 1:
                j = 0

            players_detections.class_id[i] = j

    longest_x = 0 # Initialize to min distance
    shortest_x = 12000 # Initialize to max distance
    longest_x_id = 0
    shortest_x_id = 0

    # Find the player furthest to the right and their id
    for i in range(len(pitch_players_xy)):
        if pitch_players_xy[i][0] > longest_x:
            longest_x = pitch_players_xy[i][0]
            longest_x_id = i

    # Find the player furthest to the left and their id
    for j in range(len(pitch_players_xy)):
        if pitch_players_xy[j][0] < shortest_x:
            shortest_x = pitch_players_xy[j][0]
            shortest_x_id = j
    
    offside_ids = []

    # Get furthest left and right for each team
    # Since team 0 is always left --> if team 0 isnt furthest left then team 1 is offsides
    # Since team 1 is always right --> if team 1 isnt furthest right then team 0 is offsides
    if players_detections.class_id[shortest_x_id] != 0:
        offside_ids.append(shortest_x_id)
    
    if players_detections.class_id[longest_x_id] != 1:
        offside_ids.append(longest_x_id)

    new_class_ids = players_detections.class_id

    # Loop through all offside ids and set to OFFSIDE_ID
    for i in offside_ids:
        new_class_ids[i] = OFFSIDE_ID

    # Return new list of class ids including the offside players
    return new_class_ids

In [35]:
# Transform perspective of pitch to 2D plane
class ViewTransformer:

    # Initialize points in float32 format
    # Calculate homography matrix for 2D perspective transformation
    # @Params: source<np.ndarray>, target<np.ndarray>
    # @Return: None
    def __init__(self, source: np.ndarray, target: np.ndarray):
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m, _ = cv2.findHomography(source, target)

    # Points exist in 2 dimensions
    # Expand points to 3 dimensions to match homography matrix
    # Remove extra dimension once completed
    # @Params: source<np.ndarray>, target<np.ndarray>
    # @Return: np.ndarray
    def transform_points(self, points: np.ndarray) -> np.ndarray:

        if points.size == 0:
            return points
        
        if points.shape[1] != 2:
            raise ValueError("Points must have shape (n, 2)")
        
        
        points = points.reshape(-1, 1, 2).astype(np.float32)
        points = cv2.perspectiveTransform(points, self.m)
        return points.reshape(-1, 2).astype(np.float32)

In [36]:
crops = extract_crops(SOURCE_VIDEO_PATH)
team_classifier = TeamClassifier()
team_classifier.fit(crops)

# Track newly detected instances of players/balls
tracker = sv.ByteTrack()
tracker.reset()

# Create video source and where to store completed video
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
video_sink = sv.VideoSink(TARGET_VIDEO_PATH, video_info=video_info)

# For testing only grab a single frame and draw on it for results
frame_generator = sv.get_video_frames_generator(source_path=SOURCE_VIDEO_PATH)

# Only difference between offside and offside-video is the loop to run logic on every frame of video
with video_sink:
    for frame in frame_generator:

        # Run player detection model
        player_result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
        player_detections = sv.Detections.from_inference(player_result)

        # Run pitch detection model
        pitch_result = PITCH_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
        key_points = sv.KeyPoints.from_inference(pitch_result)

        # Get detections for ball from each frame
        ball_detections = player_detections[player_detections.class_id == BALL_ID]
        ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)

        # Set all detections to not include ball
        # Remove overlapping detections
        # Adjust class id of all remaining detections by 1
        # Add tracker to all detections besides ball 
        all_detections = player_detections[player_detections.class_id != BALL_ID]
        all_detections = all_detections.with_nmm(threshold=0.5, class_agnostic=True)
        all_detections = tracker.update_with_detections(all_detections)

        
        # Define player, goalkeeper and referees from within all_detections
        players_detections = all_detections[all_detections.class_id == PLAYER_ID]
        goalkeepers_detections = all_detections[all_detections.class_id == GOALKEEPER_ID]
        referee_detections = all_detections[all_detections.class_id == REFEREE_ID]

        referee_detections.class_id -= 1

        # Crop images of each player and predict their team
        player_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
        players_detections.class_id = team_classifier.predict(player_crops)

        # Resolve goalkeeper teams by finding closest distance from average team position
        goalkeepers_detections.class_id = resolve_goalkeepers_team_id(players_detections, goalkeepers_detections)

        
        # Filter out keypoints with confidence less than 0.5
        # Create new object with filtered key points
        # Create new KeyPoints object with new key point positions
        # Filter out low confidence pitch reference points
        filter = key_points.confidence[0] > 0.5
        frame_reference_points = key_points.xy[0][filter]
        frame_reference_key_points = sv.KeyPoints(xy=frame_reference_points[np.newaxis, ...])
        pitch_reference_points = np.array(CONFIG.vertices)[filter]

        # Pass pitch_reference_Points and frame_reference_points to do transformation
        view_transformer = ViewTransformer(
            source=pitch_reference_points,
            target=frame_reference_points,    
        )

        # Gather all the points together
        # Create new KeyPoints object
        pitch_all_points = np.array(CONFIG.vertices)
        frame_all_points = view_transformer.transform_points(pitch_all_points)
        frame_all_key_points = sv.KeyPoints(xy=frame_all_points[np.newaxis, ...])

        # 2D Plane X is from 0 - 12000
        # Pass pitch_reference_Points and frame_reference_points to do transformation
        view_transformer = ViewTransformer(
            source=frame_reference_points,
            target=pitch_reference_points,    
        )

        # Get position of ball on a frame
        # Frame (x,y) is captured image
        # Pitch (x,y) is position of ball on 2D plane
        # Transform position to 2D plane
        # Same for players and referees
        frame_ball_xy = ball_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
        pitch_ball_xy = view_transformer.transform_points(frame_ball_xy)

        frame_player_xy = players_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
        pitch_players_xy = view_transformer.transform_points(frame_player_xy)

        frame_referee_xy = referee_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
        pitch_referee_xy = view_transformer.transform_points(frame_referee_xy)

        # Detect Offside Players
        # Get list of ids -- index of offside player will be 4
        # Store new list of ids in players_detections class ids
        player_detections.class_id = resolve_offside_player(players_detections, pitch_players_xy)

        # Redefine all_detections to merge players and goalkeepers
        all_detections = sv.Detections.merge([players_detections, goalkeepers_detections, referee_detections])

        # Ensure class_id values are integers
        all_detections.class_id = np.array(all_detections.class_id, dtype=int)

        # Update tracker label
        # Add id to each object being tracked
        labels = [
            "Offsides" if class_id == 4 else f"#{tracker_id}"
            for tracker_id, class_id in zip(all_detections.tracker_id, all_detections.class_id)
        ]

        # Set annotated frame to current frame
        # Add ellipise & label annotator to all detections (contains players & goalkeepers)
        # Add triangle annotator to ball detections
        annotated_frame = frame.copy()
        annotated_frame = ellipse_annotator.annotate(annotated_frame, all_detections)
        annotated_frame = label_annotator.annotate(annotated_frame, all_detections, labels=labels)
        annotated_frame = triangle_annotator.annotate(annotated_frame, ball_detections)

        # Plot single frame image for testing results
        video_sink.write_frame(annotated_frame)



collection crops: 18it [00:03,  5.27it/s]
Embedding extraction: 11it [00:37,  3.43s/it]
Embedding extraction: 1it [00:02,  2.09s/it]


[[3970.4976 6403.346 ]
 [4995.2417 6860.88  ]
 [3856.0762 4196.644 ]
 [4870.521  4651.581 ]
 [6179.0156 4542.7134]
 [4810.893  3068.1885]
 [3810.7441 1224.2502]
 [6801.3345 3616.7004]
 [6765.1997 4780.9077]
 [2784.6294 4610.7446]
 [6695.631   271.0819]]
Offside IDS:  []


Embedding extraction: 1it [00:02,  2.18s/it]


[[3975.266  6413.7866]
 [4994.3086 6866.523 ]
 [4869.969  4652.5215]
 [3859.411  4199.063 ]
 [6173.5337 4542.2363]
 [4813.3354 3070.153 ]
 [2797.1208 4613.64  ]
 [6769.8823 4762.026 ]
 [6799.684  3616.3423]
 [3825.6904 1267.1986]
 [6696.4736  272.2872]]
Offside IDS:  []


Embedding extraction: 1it [00:02,  2.34s/it]


...

[[5656.812  4104.2515]
 [7443.7075 5000.89  ]
 [6436.419  4853.815 ]
 [5594.9316 2338.3801]
 [8121.578  6189.6914]
 [6780.7817 5866.8076]
 [8982.656  4089.9712]
 [6096.219  6261.465 ]
 [6147.157  5927.6465]
 [6107.0083 5817.9004]]
Offside IDS:  [13]
Player 13 is offside


Embedding extraction: 1it [00:02,  2.07s/it]


[[7432.5933 4994.4023]
 [5639.636  4080.29  ]
 [6754.301  5897.0283]
 [5591.328  2337.9602]
 [8968.911  4077.525 ]
 [8119.334  6174.042 ]
 [6409.4263 4863.8413]
 [6141.816  5927.056 ]
 [6069.094  6168.836 ]
 [6102.693  5585.976 ]]
Offside IDS:  [10]
Player 10 is offside


Embedding extraction: 1it [00:02,  2.14s/it]


[[6721.852  5956.6943]
 [6040.5615 6207.624 ]
 [5596.6216 2328.918 ]
 [5629.2437 4073.6626]
 [7406.084  5023.156 ]
 [6382.4663 4874.6484]
 [8099.7524 6256.8657]
 [8933.323  4105.418 ]
 [6131.133  5966.908 ]
 [6095.258  5594.4775]]
Offside IDS:  [13]
Player 13 is offside
